In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
driver = webdriver.Chrome()
url = 'https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0'
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [16]:
prod_items = soup.select('li.prod_item')
print(len(prod_item))

43


In [17]:
title = prod_items[0].select('p.prod_name > a')[0].get_text().strip()
title

'베이스어스 차량용 청소기 A3 (해외구매)'

In [18]:
# 스펙
spec_list = prod_items[0].select('div.spec_list')[0].text.replace("\t", '').replace("\n", '')
spec_list

'차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm'

In [19]:
# 가격
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
price

'47510'

In [24]:
# 1페이지에 대한 상품정보 추출
prod_data = []
for prod_item in prod_items:
    if 'ad_nvista' in prod_item.select_one('div')['class']:
        continue
    try:
        title = prod_item.select('p.prod_name > a')[0].get_text().strip()
        spec_list = prod_item.select('div.spec_list')[0].text.replace("\t", '').replace("\n", '')
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
        prod_data.append([title, spec_list, price])
    except:
        pass
print(len(prod_data))
prod_data

40


[['베이스어스 차량용 청소기 A3 (해외구매)',
  '차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm',
  '47510'],
 ['LG전자 오브제컬렉션 코드제로 A9S AU9982',
  '핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이드바닥 / 스팀물걸레 / 침구 / 솔형 / 틈새 / 올인원타워 / [배터리] 분리형 (2개) / 충전시간: 4시간 / 사용시간: 1시간 / [성능] 스마트인버터모터 / 헤파필터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 자동먼지비움 / [부가] 색상: 카밍베이지 / 무게: 2.7kg / 크기(가로x세로x깊이): 300x1120x260mm',
  '1026090'],
 ['샤오미 미지아 차량용 핸디 무선 청소기 (해외구매)',
  '차량용청소기 / 무선 / 흡입력: 13,000Pa / 최대출력: 120W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / 사용시간: 9~30분 / 버튼 잠금 장치 / 2단 세기 조절 / 크기: 55x267mm / 무게: 500g',
  '38990'],
 ['LG전자 오브제컬렉션 코드제로 A9S AU9882',
  '핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이드바닥 / 스팀물걸레 / 솔형 / 틈새 / 올인원타워 / [배터리] 분리형 (2개) / 충전시간: 4시간 / 사용시간: 1시간 / [성능] 스마트인버터모터 / 헤파필터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 자동먼지비움 / [부가] 색상: 카밍베이지 / 무게: 2.7kg / 크기(가로x세로x깊이): 300x1120x260mm

In [25]:
# 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'ad_nvista' in prod_item.select_one('div')['class']:
            continue
        try:
            title = prod_item.select('p.prod_name > a')[0].get_text().strip()
            spec_list = prod_item.select('div.spec_list')[0].text.replace("\t", '').replace("\n", '')
            price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
            prod_data.append([title, spec_list, price])
        except:
            pass
    return  prod_data

In [26]:
prod_data = get_prod_items(prod_items)
len(prod_data)

40

# 2.여러페이지에 걸친 다나와 검색 페이지 크롤링

In [27]:
def get_search_page_url(keyword, page):
    return 'https://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&mode=simple&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=3014&defaultVaTab=969662&tab=goods'.format(keyword, page)

In [28]:
keyword = '무선청소기'
page = 10
url =  get_search_page_url(keyword, page)
print(url)

https://search.danawa.com/dsearch.php?query=무선청소기&volumeType=allvs&page=10&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&mode=simple&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=3014&defaultVaTab=969662&tab=goods


In [29]:
# 전체 페이지 코드정리
keyword = '무선청소기'
total_page = 10
prod_data_total = []
for page in tqdm(range(1, total_page+1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(4)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    #상품정보 추출
    prod_items = soup.select('li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    # 추출한 정보 저장
    prod_data_total += prod_item_list

  0%|          | 0/10 [00:00<?, ?it/s]

In [31]:
df = pd.DataFrame(prod_data_total, columns=['상품명', '스펙목록', '가격'])
df.tail()

,상품명,스펙목록,가격
395,유피 무선 차량 청소기 H20,"차량용청소기 / 무선 / 흡입력: 14,000Pa / 추가노즐: 틈새, 솔브러쉬 /...",125390
396,삼성전자 비스포크 제트 VS20A957E2,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 210W / [구성] 바닥...,708990
397,삼성전자 제트 VS20T9253QD,핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 200W / [구성] 바닥 / 솔...,417600
398,샤오미 미홀 H17 Pro,물걸레청소기 / 롤러식 / 물걸레+진공 / 무선형 / [배터리] 리튬이온 (분리형)...,215100
399,LG전자 오브제컬렉션 코드제로 A9S AT9271,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 210W / [구성] 바닥...,875970


In [32]:
df.to_csv('danawa_crawling_result.csv', index=False)